In [1]:
import sqlite3
import pandas as pd
import numpy as np
from math import log

# Step 1: Connect to the SQLite database and load data into Pandas DataFrames
conn = sqlite3.connect('PBS.db')

# Load data from builder_proposer_tx and relay_data tables
builder_proposer_tx = pd.read_sql_query("SELECT * FROM builder_proposer_tx", conn)
relay_data = pd.read_sql_query("SELECT * FROM relay_data", conn)

# Step 2: Perform the equivalent operations as in the SQL query using Pandas
# Join builder_proposer_tx and relay_data on block_hash
merged_data = pd.merge(builder_proposer_tx, relay_data.drop_duplicates(subset='block_number'), on='block_hash')

merged_data




,hash,nonce,transaction_index,from_address,to_address,value_x,gas,gas_price,input,receipt_cumulative_gas_used,...,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,relay,slot,builder_pubkey,value_y,gas_used,gas_limit,block_number_y
0,0x0f279685290200812f28fa72ee6003d5a81e00188e30...,463322,113,0x1f9090aae28b8a3dceadf281b0f12828e676c326,0xb3d9cf8e163bbc840195a97e81f8a34e295b8f39,9.334251e+16,29683,16675160900,0x,9773650,...,0.0,2,16675160900,agnostic Gnosis,8146863,0x978a35c39c41aadbe35ea29712bccffb117cc6ebcad4...,93342509421662937,9773650,30000000,18951601
1,0xf06662ce48394fecc8f7f8caeba921c85b247a1b72b7...,12189,170,0x0aa8ebb6ad5a8e499e550ae2c461197624c6e667,0x0000000000450702bc4f750fd1e7ecad7054c4f1,2.114301e+16,50451,12555993581,0x000000000adc0aa8ebb6ad5a8e499e550ae2c4611976...,12097702,...,0.0,2,12555993581,flashbots,8147189,0xae7f0dd473d986b277539cca610aba02c437082ac504...,21143014533381004,12097702,30000000,18951918
2,0x943daccf8e4098b949c01a4af2192ee08423faa1f080...,24991,143,0xbd3afb0bb76683ecb4225f9dbc91f998713c3b01,0x717bbc4fa6574abb4ffadccfe00270be21e7473a,1.447291e+17,55000,13241336347,0x,13710231,...,0.0,2,13241336347,flashbots,8147351,0x82ba7cadcdfc1b156ba2c48c1c627428ba917858e62c...,144729125284399692,13710231,30000000,18952077
3,0xa42a9b46592d09671b5c066fd224cd198c8b350f551f...,836,5,0x229b8325bb9ac04602898b7e8989998710235d5f,0x388c818ca8b9251b393131c08a736a67ccb19297,1.667604e+16,32111,28357565471,0x,1287501,...,0.0,2,28357565471,flashbots,8148438,0x81476ce973b358648ef2b990ed3fe9e9b264030116e5...,16676039455691635,1287501,30000000,18953144
4,0x583057491a2add31540a80f54838ef0e2aa9031c9dcd...,25002,104,0xbd3afb0bb76683ecb4225f9dbc91f998713c3b01,0x13f2241aa64bb6da2b74553fa9e12b713b74f334,9.483185e+16,55000,30819362275,0x,9709848,...,0.0,2,30819362275,flashbots,8148719,0x82ba7cadcdfc1b156ba2c48c1c627428ba917858e62c...,94831853087048970,9709848,30000000,18953421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140349,0x1a3244dae7fdc997041ef808ddf480352976cff57942...,452793,141,0x1f9090aae28b8a3dceadf281b0f12828e676c326,0xd6e4aa932147a3fe5311da1b67d9e73da06f9cef,2.733885e+16,28159,8825600040,0x,19237635,...,0.0,2,8825600040,agnostic Gnosis,8106022,0x83d3495a2951065cf19c4d282afca0a635a39f6504bd...,27338848635781920,19237635,30000000,18911286
140350,0x3c4de2aad75317f6f6abfec62463443357945d8d5edd...,19966,118,0x88c6c46ebf353a52bdbab708c23d0c81daa8134a,0xd6e4aa932147a3fe5311da1b67d9e73da06f9cef,2.055684e+16,28159,20435446364,0x,12369992,...,0.0,2,20435446364,aestus,8286466,0xb0a3a5e8e0330698176d35fda999f522d99222597559...,20556844667292142,12369992,30000000,19089909
140351,0x2e625da775e396ab6b04a75c5b44b9f46941bb1d0a9c...,639280,167,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0xd6e4aa932147a3fe5311da1b67d9e73da06f9cef,1.610417e+16,28159,23122519806,0x,19237776,...,0.0,2,23122519806,ultrasound,8288089,0xaec4ec48c2ec03c418c599622980184e926f0de3c9ce...,16104174763309807,19237776,30000000,19091524
140352,0x96a0f4f8ebe1cc7adf41eab10492b57baf9dedaeecc2...,491427,216,0x1f9090aae28b8a3dceadf281b0f12828e676c326,0xd6e4aa932147a3fe5311da1b67d9e73da06f9cef,7.989162e+16,28159,16546507781,0x,25153398,...,0.0,2,16546507781,flashbots,8289757,0x978a35c39c41aadbe35ea29712bccffb117cc6ebcad4...,79891616993233902,25153398,30000000,19093173


In [2]:
address_counts = merged_data.groupby('relay')['block_hash'].nunique().reset_index(name='address_count')

# Step 4: Calculate TotalCount
total_count = merged_data['block_hash'].nunique()


In [3]:
# Step 5: Calculate Shannon entropy
address_counts['probability'] = address_counts['address_count'] / total_count
address_counts['shannon_term'] = -1.0 * address_counts['probability'] * np.log(address_counts['probability'])
shannon_entropy = address_counts['shannon_term'].sum()
shannon_entropy

1.225888778097095

In [4]:
total_count

140354

In [5]:
address_counts

,relay,address_count,probability,shannon_term
0,aestus,5903,0.042058,0.133269
1,agnostic Gnosis,26792,0.190889,0.316124
2,eden,609,0.004339,0.023605
3,flashbots,56257,0.400822,0.366447
4,manifold,458,0.003263,0.018682
5,ultrasound,50335,0.358629,0.367762
